In [ ]:
import os
import utils
import connectome_create
# viz_method = one of ['itkwidgets', 'vtk']
viz_method = 'vtk'

# import some of our favorite packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings

%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
# import nglui.statebuilder as ngstbld

# # this is the EM specific package for querying the EM data
from caveclient import CAVEclient

from sklearn.decomposition import PCA
from numpy.random import default_rng

# from meshparty import trimesh_io, trimesh_vtk
# from meshparty import skeletonize, skeleton_io, skeleton
# import cloudvolume

%load_ext autoreload
%autoreload 2

In [ ]:
# from pathlib import Path
# import json

# with open(Path.home() / '.cloudvolume/secrets/'/'cave-secret.json') as f:
#         tokens = json.load(f)
        
# seg_source = 'graphene://https://cave.fanc-fly.com/segmentation/table/mar2021_prod'
# cv = cloudvolume.CloudVolume(cloudpath=seg_source, use_https=True, secrets=tokens)

In [ ]:
datastack_name = 'fanc_production_mar2021'

client = CAVEclient(datastack_name)

In [ ]:
# client = CAVEclient()

# # if not os.path.isfile(os.path.expanduser("~/.cloudvolume/secrets/cave-secret.json")):
# client.auth.get_new_token(open=True)

In [ ]:
# if you have not yet setup this computer, uncomment this below line
# paste the token from the website in, and run the line

# client.auth.save_token(token="c14cd7a3e18a1a697716a399afbf5778", overwrite=True)

# then comment or delete the line as you don't need to run it on this computer  again

In [ ]:
pre_to_mn_df = connectome_create.load_pre_to_mn_df(ext='matched_typed_with_nt')
pre_to_mn_df.shape

# Figure 2
1. Connectivity matrix
2. Numbers of each class

In [ ]:
lbls = utils.mn_labels(pre_to_mn_df.columns.to_frame(),depth='rank')


fig = plt.figure(1, figsize = [8,20])
import cmocean
cmap_alt = cmocean.cm.dense
cmap = utils.white_dense()
ax = sns.heatmap(np.log10(pre_to_mn_df.to_numpy()+1), xticklabels=lbls, cmap=cmap)
ax.set_rasterized(True)
cbar = ax.collections[0].colorbar
# here set the labelsize by 20
# cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'log 10 # of synapses', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
# plt.ylabel(ylabelstring, fontsize =18)
# plt.yticks(fontsize = 16)
# plt.xticks(fontsize = 16)
# plt.show()

# plt.xlabel(None)
# plt.ylabel(None)
# plt.xticks([])
# plt.yticks([])
plt.show()


fig.savefig('./figpanels/matched_premotor_to_motor.eps',format='eps',dpi=300, bbox_inches='tight')
# plt.savefig('wingConnMatrix0110.eps', format='eps', bbox_inches='tight')
# fig.savefig('./figpanels/matched_premotor_to_motor.png',format='png')

## Number of partners

In [ ]:
# total number of synapses from actual neurons (no fragments)
pre_to_mn_df.loc[(['local','intersegmental','ascending','sensory','descending'])].sum(axis=0).max()

In [ ]:
# Double check the order of the neurons
x = utils.xticks_from_pools(pre_to_mn_df.columns.to_frame())
lbls = utils.mn_labels(pre_to_mn_df.columns.to_frame(),depth='rank')
mat = pre_to_mn_df.to_numpy()
y = pre_to_mn_df.astype(bool).sum(axis=0).to_numpy()

fig = plt.figure(1, figsize=(20, 6))
ax1 = plt.subplot2grid((1,1),(0,0))

ax1.scatter(x, y,marker='o')
plt.sca(ax1)
plt.title('Number of descending neurons connected to each MN')
plt.xlabel('MN')
plt.ylabel('Number of inputs')
locs, labels = plt.xticks(ticks=x, labels=lbls, rotation=90)

ax1.set_ylim([0, 400])

plt.savefig('./figpanels/total_partners.svg',format='svg')

# mean number of partners
y.mean()
# max number of partners
# y.max()

## Number of targets per preMN

In [ ]:
pre_to_mn_df.head()
partners = pre_to_mn_df.loc[(['local','intersegmental','ascending','sensory','descending'])].astype(bool).groupby(by='cell_class',axis=0).sum()
# mean numbers of targets partners
y = pre_to_mn_df.astype(bool).sum(axis=1).to_numpy()

x = [i for i in range(len(pre_to_mn_df.index.to_frame()['cell_class'].values))]
lbls = pre_to_mn_df.index.to_frame()['cell_class'].values

fig = plt.figure(1, figsize=(20, 6))
ax1 = plt.subplot2grid((1,1),(0,0))

ax1.scatter(x, y,marker='o')
plt.sca(ax1)
plt.title('Number of MN targets per preMN')
plt.xlabel('MN')
plt.ylabel('Number of targets')
locs, labels = plt.xticks(ticks=x, labels=lbls, rotation=90)

# ax1.set_ylim([0, 400])

plt.savefig('./figpanels/total_partners.svg',format='svg')

# mean number of partners
y.mean()
# max number of partners
# y.max()

In [ ]:
mn_mi = partners.columns.to_frame()
x = utils.xticks_from_mnmi(mn_mi)

partners = pre_to_mn_df.loc[(['descending','sensory','ascending','intersegmental','local'])].astype(bool).groupby(by='cell_class',axis=0).sum()
partners = partners.div(partners.sum(axis=0))
partners = partners.loc[['descending','sensory','ascending','intersegmental','local'],:]

classlist = ['local', 'intersegmental', 'ascending', 'sensory', 'descending','fragment',]
colors = ["#7D688E",  "#CC9FCC",    "#6084CC",  "#93E5B6",  "#79D0F7",        "#dddddd", ]
cmap = sns.set_palette(sns.color_palette(colors))

ax = partners[::-1].T.plot(kind='bar', stacked=True, legend = False,width=1,cmap=cmap,figsize=(30,6)) 
plt.savefig('./figpanels/prepartners_from_cell_classes.svg',format='svg')

In [ ]:
all_pre_to_mn_df = connectome_create.load_pre_to_mn_df('pre_match_to_pool_w_fragments')

# Double check the order of the neurons
x = utils.xticks_from_mnmi(mn_mi)
x = utils.xticks_from_pools(mn_mi)
lbls = utils.mn_labels(mn_mi,depth='rank')
mat = all_pre_to_mn_df.to_numpy()
y = mat.sum(axis=0)

fig = plt.figure(1, figsize=(20, 6))
ax1 = plt.subplot2grid((1,1),(0,0))

ax1.scatter(x, y,marker='o')
plt.sca(ax1)
plt.title('Number of descending neurons connected to each MN')
plt.xlabel('MN')
plt.ylabel('Number of inputs')
locs, labels = plt.xticks(ticks=x, labels=lbls, rotation=90)

plt.savefig('./figpanels/total_inputs.svg',format='svg')

## Fraction of total input from each cell_class

In [ ]:
mn_mi = all_pre_to_mn_df.columns.to_frame()
x = utils.xticks_from_mnmi(mn_mi)

frac_df = pd.DataFrame(index=['fragment','descending','sensory','ascending','intersegmental','local','x'],columns=pre_to_mn_df.columns)
frac_df.loc['fragment',:] = all_pre_to_mn_df.loc['fragment'].sum(axis=0)
frac_df.loc['descending',:] = all_pre_to_mn_df.loc['descending'].sum(axis=0)
frac_df.loc['sensory',:] = all_pre_to_mn_df.loc['sensory'].sum(axis=0)
frac_df.loc['ascending',:] = all_pre_to_mn_df.loc['ascending'].sum(axis=0)
frac_df.loc['intersegmental',:] = all_pre_to_mn_df.loc['intersegmental'].sum(axis=0)
frac_df.loc['local',:] = all_pre_to_mn_df.loc['local'].sum(axis=0)
# frac_df.loc['motor',:] = mns.sum(axis=0)
frac_df = frac_df/frac_df.sum(axis=0)
frac_df.loc['x',:] = x

# sns.set(rc = {'figure.figsize':(15,8)})


# # locs, labels = plt.xticks(ticks=x, labels=lbls, rotation=90)
          #'motor', 'nont1',    'desc',     'sens',     'ascend',   'local',    'intr',     'vnc',      'fragments'
classlist = ['motor', 'descending', 'sensory', 'ascending', 'intersegmental','local',   'non_t1_motor', 'descending_post',  'sensory_post','ascending_post','neurmodulatory_ascending_neuron', 'vnc_non_premotor']
colors = ["#910951",  "#79D0F7",    "#93E5B6",  "#6084CC",  "#CC9FCC",        "#7D688E", "#CC4893",      '#79D0F7',          "#93E5B6",      "#6084CC",      "#CC4893",                          "#CC9FCC"]
classlist = ['fragment', 'descending', 'sensory', 'ascending', 'intersegmental','local',]
colors = ["#dddddd",  "#79D0F7",    "#93E5B6",  "#6084CC",  "#CC9FCC",        "#7D688E", ]
classlist = ['local', 'intersegmental', 'ascending', 'sensory', 'descending','fragment',]
colors = ["#7D688E",  "#CC9FCC",    "#6084CC",  "#93E5B6",  "#79D0F7",        "#dddddd", ]
cmap = sns.set_palette(sns.color_palette(colors))

# ax = frac_df.T.plot(x='x',kind='barh', stacked=True, legend = True,align='edge') 
ax = frac_df[::-1].T.plot(x='x',kind='bar', stacked=True, legend = False,width=1,cmap=cmap,figsize=(30,6)) 
# Most of them are pretty close to 1

# ax = frac_df.iloc[::-1].T.plot(x='x',kind='bar', stacked=True, legend = True) #, color=['aqua','red', 'steelblue','orange','yellow'])
# locs, labels = plt.xticks(ticks=[i for i in range(len(lbls))], labels=lbls, rotation=90)
plt.savefig('./figpanels/input_from_cell_classes.svg',format='svg')

## Total fractional input from each class 

### Pie chart  

In [ ]:
piesums = all_pre_to_mn_df.groupby(level='cell_class').sum().sum(axis=1)
piesums = piesums.loc[['local','intersegmental','motor','fragment','descending','sensory','ascending']]
print(piesums.sum())
piesums


In [ ]:
fig, ax = plt.subplots( 1, 1, figsize=(8,8))
ax.pie(piesums, explode=None, labels=piesums.index, autopct='%1.1f%%',
              shadow=False, startangle=90)

fname = './figpanels/class_input_to_mns_pie.svg'
fig.savefig(fname,format='svg')

## Total synapses

In [ ]:
print('Total synapses: {}'.format(piesums.sum()))

# Total sysnapses to FETi that are not fragments

In [ ]:
all_pre_to_mn_df.loc[(['local','intersegmental','ascending','sensory','descending'])].sum(axis=0).max()

## Now print the segids for 13A as an example, then color them in neuroglancer

In [ ]:
All = slice(None)
pmn_index = pre_to_mn_df.index.to_frame()
pmn_index_df = pmn_index.set_index(keys=['cell_class','preferred_pool','classification_system','cell_type'],drop=False)
eg13A = pmn_index_df.loc[('local',All,'D', All,),:]
pks = eg13A.preferred_pool.unique()

pool_pal = {
    'thorax_swing':'#ff4df0',
    'thorax_stance':'#00bf4d',
    'trochanter_extension':'#FCBE86',
    'trochanter_flexion':'#95B0FC',
    'femur_reductor':'#EC1C24',
    'tibia_extensor':'#DB6800',
    'main_tibia_flexor':'#203789',
    'auxiliary_tibia_flexor_E':'#009B1D',
    'ltm':'#FEDF08',
    'tarsus_depressor_noid':'#FF8D8D',
}
# print('"segmentColors": {')    
# for pl in eg13A.preferred_pool.unique():
#     print(pl)
#     # print(eg13A.loc[(All,pl,All),'segID'])
#     for s in eg13A.loc[(All,pl,All),'segID'].to_list():
#         print('"{}": "{}",'.format(s,pool_pal[pl]))
# print('"segments": {')    
# for pl in eg13A.preferred_pool.unique():
#     for s in eg13A.loc[(All,pl,All),'segID'].to_list():
#         print('"{}",'.format(s))


## All

In [ ]:
# Cluster according to everything but fragments
mn_mi = pre_to_mn_df.columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')

sim_mat = cosine_similarity(pre_to_mn_df.to_numpy().transpose())
fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(sim_mat, xticklabels=mn_mi.segID.to_list(), yticklabels=lbls,cmap=plt.get_cmap('Greys'))
ax.xaxis.set_ticks_position('top')
cbar = ax.collections[0].colorbar
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

# here set the labelsize by 20
cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'cosine similarity', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
plt.ylabel('Motor Neurons', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

fig.savefig('./figpanels/cos_sim_all_grayscale.svg',format='svg')

## Local

In [ ]:
# Cluster according to everything but fragments
mn_mi = pre_to_mn_df.columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')

sim_mat = cosine_similarity(pre_to_mn_df.loc['local',:].to_numpy().transpose())
fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(sim_mat, xticklabels=mn_mi.segID.to_list(), yticklabels=lbls,cmap=plt.get_cmap('Greys'))
ax.xaxis.set_ticks_position('top')
cbar = ax.collections[0].colorbar
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

# here set the labelsize by 20
cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'cosine similarity', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
plt.ylabel('Motor Neurons', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

fig.savefig('./figpanels/cos_sim_local_grayscale.svg',format='svg')

## Intersegmental

In [ ]:
# Cluster according to everything but fragments
mn_mi = pre_to_mn_df.columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')

sim_mat = cosine_similarity(pre_to_mn_df.loc['intersegmental',:].to_numpy().transpose())
fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(sim_mat, xticklabels=mn_mi.segID.to_list(), yticklabels=lbls,cmap=plt.get_cmap('Greys'))
ax.xaxis.set_ticks_position('top')
cbar = ax.collections[0].colorbar
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

# here set the labelsize by 20
cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'cosine similarity', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
plt.ylabel('Motor Neurons', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

fig.savefig('./figpanels/cos_sim_intersegmental_grayscale.svg',format='svg')

## Descending

In [ ]:
# Cluster according to everything but fragments
mn_mi = pre_to_mn_df.columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')

sim_mat = cosine_similarity(pre_to_mn_df.loc['descending',:].to_numpy().transpose())
fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(sim_mat, xticklabels=mn_mi.segID.to_list(), yticklabels=lbls,cmap=plt.get_cmap('Greys'))
ax.xaxis.set_ticks_position('top')
cbar = ax.collections[0].colorbar
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

# here set the labelsize by 20
cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'cosine similarity', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
plt.ylabel('Motor Neurons', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

fig.savefig('./figpanels/cos_sim_descending_grayscale.svg',format='svg')

## Ascending

In [ ]:
# Cluster according to everything but fragments
mn_mi = pre_to_mn_df.columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')

sim_mat = cosine_similarity(pre_to_mn_df.loc['ascending',:].to_numpy().transpose())
fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(sim_mat, xticklabels=mn_mi.segID.to_list(), yticklabels=lbls,cmap=plt.get_cmap('Greys'))
ax.xaxis.set_ticks_position('top')
cbar = ax.collections[0].colorbar
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

# here set the labelsize by 20
cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'cosine similarity', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
plt.ylabel('Motor Neurons', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

fig.savefig('./figpanels/cos_sim_ascending_grayscale.svg',format='svg')

## Sensory

In [ ]:
# Cluster according to everything but fragments
mn_mi = pre_to_mn_df.columns.to_frame()
lbls = utils.mn_labels(mn_mi,depth='rank')

sim_mat = cosine_similarity(pre_to_mn_df.loc['sensory',:].to_numpy().transpose())
fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(sim_mat, xticklabels=mn_mi.segID.to_list(), yticklabels=lbls,cmap=plt.get_cmap('Greys'))
ax.xaxis.set_ticks_position('top')
cbar = ax.collections[0].colorbar
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

# here set the labelsize by 20
cbar.ax.tick_params(labelsize=20)
cbar.set_label(label = 'cosine similarity', size=24)
plt.xlabel('Motor Neurons', fontsize =18)
plt.ylabel('Motor Neurons', fontsize =18)
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
plt.show()

fig.savefig('./figpanels/cos_sim_sensory_grayscale.svg',format='svg')

## Regress local vs. normal, vs. descending, etc.

In [ ]:
mn_mi_df = pre_to_mn_df.columns.to_frame()
mn_mi_df

# Ha! for this, put the tarsus neurons in the right pools
muscle_tuple_dict = utils.get_motor_pool_tuple_dict()
pool_keys = [
    'thorax_swing',
    'thorax_stance',
    'trochanter_extension',
    'trochanter_flexion',
    'femur_reductor',
    'tibia_extensor',
    'main_tibia_flexor',
    # 'auxiliary_tibia_flexor_A',
    'auxiliary_tibia_flexor_B',
    'auxiliary_tibia_flexor_E',
    'ltm',
    'tarsus_depressor',
    ]


    #  'main_tibia_flexor_wtarsus':                 
    #  'auxiliary_tibia_flexor_B_wtarsus':
    #  'auxiliary_tibia_flexor_E_wtarsus':
    #             'tarsus_depressor_medial':
    #             'tarsus_depressor_ventralU':  
pool_keys = [
    'thorax_swing',
    'thorax_stance',
    'trochanter_extension',
    'trochanter_flexion',
    'femur_reductor',
    'tibia_extensor',
    'main_tibia_flexor_wtarsus',
    # 'auxiliary_tibia_flexor_A',
    'auxiliary_tibia_flexor_B_wtarsus',
    'auxiliary_tibia_flexor_E_wtarsus',
    'ltm',
    'tarsus_depressor_medial',
    'tarsus_depressor_ventralU'
    ]


for key in pool_keys:
    mn_tup = muscle_tuple_dict[key]
    mn_mi_df.loc[mn_tup,'preferred_pool'] = key

mn_mi_df.preferred_pool = mn_mi_df.preferred_pool.astype("category")
mn_mi_df.preferred_pool = mn_mi_df.preferred_pool.cat.set_categories(pool_keys)
mn_mi_df

In [ ]:
sim_mat_all = cosine_similarity(pre_to_mn_df.to_numpy().transpose())
sim_mat_local = cosine_similarity(pre_to_mn_df.loc['local',:].to_numpy().transpose())
sim_mat_desc = cosine_similarity(pre_to_mn_df.loc['descending',:].to_numpy().transpose())
sim_mat_inter = cosine_similarity(pre_to_mn_df.loc['intersegmental',:].to_numpy().transpose())
sim_mat_ascending = cosine_similarity(pre_to_mn_df.loc['ascending',:].to_numpy().transpose())
sim_mat_sensory = cosine_similarity(pre_to_mn_df.loc['sensory',:].to_numpy().transpose())

tril_mask = np.full((sim_mat_all.shape[0], sim_mat_all.shape[1]), True, dtype=bool)
motor_pool_mask = np.full((sim_mat_all.shape[0], sim_mat_all.shape[1]), False, dtype=bool)
i_index = np.full((sim_mat_all.shape[0], sim_mat_all.shape[1]), 0, dtype=int)
j_index = np.full((sim_mat_all.shape[0], sim_mat_all.shape[1]), 0, dtype=int)
for i in range(motor_pool_mask.shape[0]):
    for j in range(motor_pool_mask.shape[1]):
        motor_pool_mask[i,j] =(mn_mi_df.iloc[i,:].preferred_pool==mn_mi_df.iloc[j,:].preferred_pool)
        i_index[i,j] = i
        j_index[i,j] = j

fig = plt.figure(1, figsize = [6,6])
ax = sns.heatmap(motor_pool_mask, xticklabels=mn_mi_df.preferred_pool.to_list(),cmap=plt.get_cmap('Greys'))

# fig = plt.figure(2, figsize = [6,6])
# ax = sns.heatmap(np.tril(motor_pool_mask,-1), xticklabels=mn_mi_df.preferred_pool.to_list(),cmap=plt.get_cmap('Greys'))

tril_mask = np.tril(tril_mask,-1)

pool_arr = mn_mi_df.preferred_pool.to_numpy()
pool_arr = np.tile(pool_arr.reshape((-1,1)),len(pool_arr))
pool_arr.reshape(-1)

In [ ]:
sim_dict = {
    'all_comps':sim_mat_all.reshape(-1),
    'local':sim_mat_local.reshape(-1),
    'desc':sim_mat_desc.reshape(-1),
    'inter':sim_mat_inter.reshape(-1),
    'asce':sim_mat_ascending.reshape(-1),
    'sense':sim_mat_sensory.reshape(-1),
    'pool':pool_arr.reshape(-1),
    'same_pool': motor_pool_mask.reshape(-1),
    'lower_tri':tril_mask.reshape(-1),
    'i_index':i_index.reshape(-1),
    'j_index':j_index.reshape(-1)
    }


In [ ]:

sim_compare_df = pd.DataFrame.from_dict(data = sim_dict,orient='columns')
sim_compare_df
# # get rid of upper triangle
sim_compare_df = sim_compare_df.loc[sim_compare_df.lower_tri.to_numpy()]
sim_compare_df


# Rain cloud plots if within vs across pool cosine similarity

In [ ]:
all(sim_compare_df.lower_tri)
sim_compare_df

In [ ]:
(~sim_compare_df['same_pool']).sum()+sim_compare_df['same_pool'].sum()

In [ ]:
# f, ax = plt.subplots(figsize=(7, 7))
# # seaborn.violinplot(data=pprst, *, x=None, y=None, hue=None, order=None, hue_order=None, bw='scott', cut=2, scale='area', scale_hue=True, gridsize=100, width=0.8, inner='box', split=False, dodge=True, orient=None, linewidth=None, color=None, palette=None, saturation=0.75, ax=None, **kwargs)
# seaborn.violinplot(data=pprst,x='NT',y='pref_perc', orient='vertical',scale='count', cut=0, ax=ax)
# # sns.stripplot(data=pprst,x='NT',y='pref_perc',hue='syn_total',ax=ax)


# f, ax = plt.subplots(figsize=(7, 7))
# sns.violinplot(data=sim_compare_df,x='same_pool',y='all_comps',orient='vertical',scale='area', cut=0, ax=ax)

# sim_same_pool = sim_compare_df.loc[sim_compare_df.same_pool,:]
sim_same_pool_stacked = sim_compare_df.loc[:,['all_comps','pool','same_pool','lower_tri']]
sim_same_pool_stacked['comp_cat'] = 'all_comps'
sim_same_pool_stacked = sim_same_pool_stacked.rename({'all_comps':'cos_sim'},axis='columns')

block = sim_compare_df.loc[:,['local','pool','same_pool','lower_tri']]
block['comp_cat'] = 'local'
block = block.rename({'local':'cos_sim'},axis='columns')

sim_same_pool_stacked = pd.concat([sim_same_pool_stacked,block])
sim_same_pool_stacked

block = sim_compare_df.loc[:,['inter','pool','same_pool','lower_tri']]
block['comp_cat'] = 'inter'
block = block.rename({'inter':'cos_sim'},axis='columns')

sim_same_pool_stacked = pd.concat([sim_same_pool_stacked,block])
sim_same_pool_stacked

block = sim_compare_df.loc[:,['desc','pool','same_pool','lower_tri']]
block['comp_cat'] = 'desc'
block = block.rename({'desc':'cos_sim'},axis='columns')

sim_same_pool_stacked = pd.concat([sim_same_pool_stacked,block])

block = sim_compare_df.loc[:,['asce','pool','same_pool','lower_tri']]
block['comp_cat'] = 'asce'
block = block.rename({'asce':'cos_sim'},axis='columns')

sim_same_pool_stacked = pd.concat([sim_same_pool_stacked,block])

block = sim_compare_df.loc[:,['sense','pool','same_pool','lower_tri']]
block['comp_cat'] = 'sense'
block = block.rename({'sense':'cos_sim'},axis='columns')

sim_same_pool_stacked = pd.concat([sim_same_pool_stacked,block])
sim_same_pool_stacked

# ax.set_ylim([0,1.1])
# f.savefig('./figpanels/cosine_sim_violin_all_comps.svg',format='svg')

# ax.set_ylim([0,1.1])

# f.savefig('./figpanels/pool_pref_by_NT_violin.svg',format='svg')

f, ax = plt.subplots(figsize=(7, 7))
sns.violinplot(data=sim_same_pool_stacked.loc[sim_same_pool_stacked.same_pool],x='comp_cat',y='cos_sim',orient='vertical',scale='area', cut=0, ax=ax,bw=.25)

f.savefig('./figpanels/cosine_sim_violin_same_pool.svg',format='svg')

f, ax = plt.subplots(figsize=(7, 7))
sns.violinplot(data=sim_same_pool_stacked.loc[~sim_same_pool_stacked.same_pool,:],x='comp_cat',y='cos_sim',orient='vertical',scale='area', cut=0, ax=ax,bw=.25)

f.savefig('./figpanels/cosine_sim_violin_different_pools.svg',format='svg')


f, ax = plt.subplots(figsize=(7, 7))
sns.stripplot(data=sim_same_pool_stacked.loc[sim_same_pool_stacked.same_pool],x='comp_cat',y='cos_sim',ax=ax)

f.savefig('./figpanels/cosine_sim_rain_same_pool.svg',format='svg')

f, ax = plt.subplots(figsize=(7, 7))
sns.stripplot(data=sim_same_pool_stacked.loc[~sim_same_pool_stacked.same_pool,:],x='comp_cat',y='cos_sim',ax=ax)

f.savefig('./figpanels/cosine_sim_rain_different_pools.svg',format='svg')

In [ ]:
# Stats

In [ ]:
sim_w_pool = sim_compare_df.loc[sim_compare_df['same_pool'],:]
sim_w_pool.melt(value_vars=['all_comps','local','desc','inter'])

In [ ]:
import scipy
scipy.stats.kruskal(sim_w_pool['all_comps'],
                    sim_w_pool['local'],
                    sim_w_pool['desc'],
                    sim_w_pool['inter'],)
# scipy.stats.kruskal(ratiodf.iloc[:,0],ratiodf.iloc[:,1],ratiodf.iloc[:,2],ratiodf.iloc[:,3],ratiodf.iloc[:,4],ratiodf.iloc[:,5],ratiodf.iloc[:,6],ratiodf.iloc[:,7],ratiodf.iloc[:,8])

In [ ]:
import scikit_posthocs 

scikit_posthocs.posthoc_conover(sim_w_pool.melt(value_vars=['all_comps','local','desc','inter']),val_col= 'value',group_col='variable',p_adjust='holm')


In [ ]:
# f, ax = plt.subplots(figsize=(7, 7))
# # seaborn.violinplot(data=pprst, *, x=None, y=None, hue=None, order=None, hue_order=None, bw='scott', cut=2, scale='area', scale_hue=True, gridsize=100, width=0.8, inner='box', split=False, dodge=True, orient=None, linewidth=None, color=None, palette=None, saturation=0.75, ax=None, **kwargs)
# seaborn.violinplot(data=pprst,x='NT',y='pref_perc', orient='vertical',scale='count', cut=0, ax=ax)
# # sns.stripplot(data=pprst,x='NT',y='pref_perc',hue='syn_total',ax=ax)


# f, ax = plt.subplots(figsize=(7, 7))
# sns.violinplot(data=sim_compare_df,x='same_pool',y='all_comps',orient='vertical',scale='area', cut=0, ax=ax)

# sim_same_pool = sim_compare_df.loc[sim_compare_df.same_pool,:]
sim_same_pool_stacked = sim_compare_df.loc[:,['all_comps','pool','same_pool','lower_tri']]
sim_same_pool_stacked['comp_cat'] = 'all_comps'
sim_same_pool_stacked = sim_same_pool_stacked.rename({'all_comps':'cos_sim'},axis='columns')

block = sim_compare_df.loc[:,['local','pool','same_pool','lower_tri']]
block['comp_cat'] = 'local'
block = block.rename({'local':'cos_sim'},axis='columns')

sim_same_pool_stacked = pd.concat([sim_same_pool_stacked,block])
sim_same_pool_stacked

block = sim_compare_df.loc[:,['inter','pool','same_pool','lower_tri']]
block['comp_cat'] = 'long_range'
block = block.rename({'inter':'cos_sim'},axis='columns')

sim_same_pool_stacked = pd.concat([sim_same_pool_stacked,block])
sim_same_pool_stacked

block = sim_compare_df.loc[:,['desc','pool','same_pool','lower_tri']]
block['comp_cat'] = 'long_range'
block = block.rename({'desc':'cos_sim'},axis='columns')

sim_same_pool_stacked = pd.concat([sim_same_pool_stacked,block])
sim_same_pool_stacked

# ax.set_ylim([0,1.1])
# f.savefig('./figpanels/cosine_sim_violin_all_comps.svg',format='svg')

# ax.set_ylim([0,1.1])

# f.savefig('./figpanels/pool_pref_by_NT_violin.svg',format='svg')

f, ax = plt.subplots(figsize=(7, 7))
sns.violinplot(data=sim_same_pool_stacked.loc[sim_same_pool_stacked.same_pool],x='comp_cat',y='cos_sim',orient='vertical',scale='area', cut=0, ax=ax,bw=.25)

f.savefig('./figpanels/cosine_sim_violin_same_pool_long.svg',format='svg')

f, ax = plt.subplots(figsize=(7, 7))
sns.violinplot(data=sim_same_pool_stacked.loc[~sim_same_pool_stacked.same_pool,:],x='comp_cat',y='cos_sim',orient='vertical',scale='area', cut=0, ax=ax, bw=.3)

f.savefig('./figpanels/cosine_sim_violin_different_pools_long.svg',format='svg')


f, ax = plt.subplots(figsize=(7, 7))
sns.stripplot(data=sim_same_pool_stacked.loc[sim_same_pool_stacked.same_pool],x='comp_cat',y='cos_sim',ax=ax)

f.savefig('./figpanels/cosine_sim_rain_same_pool_long.svg',format='svg')

f, ax = plt.subplots(figsize=(7, 7))
sns.stripplot(data=sim_same_pool_stacked.loc[~sim_same_pool_stacked.same_pool,:],x='comp_cat',y='cos_sim',ax=ax)

f.savefig('./figpanels/cosine_sim_rain_different_pools_long.svg',format='svg')

# Stats

In [ ]:
# Simple version, just mean and std of same pool, not same pool
x = sim_compare_df.loc[sim_compare_df.same_pool,'local'].mean()
y = sim_compare_df.loc[sim_compare_df.same_pool,'inter'].mean()

x2 = sim_compare_df.loc[~sim_compare_df.same_pool,'local'].mean()
y2 = sim_compare_df.loc[~sim_compare_df.same_pool,'inter'].mean()


In [ ]:
import scipy

scipy.stats.ttest_rel(sim_compare_df.loc[sim_compare_df.same_pool,'local'],sim_compare_df.loc[sim_compare_df.same_pool,'all_comps'],alternative='two-sided',axis=0)

In [ ]:
sim_compare_df.loc[~sim_compare_df.same_pool,'local'].to_numpy()

# PCA

# All leg modules

In [ ]:
import utils
mpool_dict = utils.get_motor_pool_tuple_dict()
pool_keys = [
    'thorax_swing',
    'thorax_stance',
    'trochanter_extension', 
    'tergotrochanter',          
    'extracoxal_trochanter_depressor',
    'trochanter_extensor',
    'trochanter_flexion',
    'femur_reductor',
    'tibia_extensor',
    'main_tibia_flexor_wtarsus',
    'auxiliary_tibia_flexor_B_wtarsus',
    'auxiliary_tibia_flexor_E_wtarsus',
    # 'main_tibia_flexor',      # main_tibia_flexor for both main and A groups, or main_tibia_flexor_muscle for the muscle
    # 'auxiliary_tibia_flexor_B',
    # 'auxiliary_tibia_flexor_E',
    'ltm',
    'ltm_tiny_small',
    'ltm_tibia',
    'ltm_femur',
    # 'tarsus_depressor_med_venU',
    # 'tarsus_depressor_noid',
    ]

pref_pool_dict = {
    'thorax_swing': 'thorax_swing',
    'thorax_stance': 'thorax_stance',
    'trochanter_extension': 'trochanter_extension',
    'tergotrochanter': 'trochanter_extension',
    'extracoxal_trochanter_depressor' : 'trochanter_extension',
    'trochanter_extensor': 'trochanter_extension',
    'trochanter_flexion': 'trochanter_flexion',
    'femur_reductor':  'femur_reductor',
    'tibia_extensor': 'tibia_extensor',
    'main_tibia_flexor_wtarsus': 'main_tibia_flexor',
    'auxiliary_tibia_flexor_B_wtarsus': 'auxiliary_tibia_flexor_B',
    'auxiliary_tibia_flexor_E_wtarsus':  'auxiliary_tibia_flexor_E',
    'ltm': 'ltm',
    'ltm_tiny_small': 'ltm',
    'ltm_tibia': 'ltm',
    'ltm_femur': 'ltm',
    #'tarsus_depressor_med_venU': ,
}
fname = './figpanels/PCA_leg_modules_pct_tr_ext_ltm_breakout.svg'
# pre_to_mn_df.head()
# mpool_dict

In [ ]:
import utils
mpool_dict = utils.get_motor_pool_tuple_dict()
pool_keys = [
    'thorax_swing',
    'thorax_stance',
    'trochanter_extension', 
    # 'tergotrochanter',          
    # 'extracoxal_trochanter_depressor',
    # 'trochanter_extensor',
    'trochanter_flexion',
    'femur_reductor',
    'tibia_extensor',
    'main_tibia_flexor_wtarsus',
    'auxiliary_tibia_flexor_B_wtarsus',
    'auxiliary_tibia_flexor_E_wtarsus',
    # 'main_tibia_flexor',      # main_tibia_flexor for both main and A groups, or main_tibia_flexor_muscle for the muscle
    # 'auxiliary_tibia_flexor_B',
    # 'auxiliary_tibia_flexor_E',
    'ltm',
    #'ltm_tiny_small',
    #'ltm_tibia',
    #'ltm_femur',
    # 'tarsus_depressor_med_venU',
    # 'tarsus_depressor_noid',
    ]

pref_pool_dict = {
    'thorax_swing': 'thorax_swing',
    'thorax_stance': 'thorax_stance',
    'trochanter_extension': 'trochanter_extension',
    'tergotrochanter': 'trochanter_extension',
    'extracoxal_trochanter_depressor' : 'trochanter_extension',
    'trochanter_extensor': 'trochanter_extension',
    'trochanter_flexion': 'trochanter_flexion',
    'femur_reductor':  'femur_reductor',
    'tibia_extensor': 'tibia_extensor',
    'main_tibia_flexor_wtarsus': 'main_tibia_flexor',
    'auxiliary_tibia_flexor_B_wtarsus': 'auxiliary_tibia_flexor_B',
    'auxiliary_tibia_flexor_E_wtarsus':  'auxiliary_tibia_flexor_E',
    'ltm': 'ltm',
    'ltm_tiny_small': 'ltm',
    'ltm_tibia': 'ltm',
    'ltm_femur': 'ltm',
    #'tarsus_depressor_med_venU': ,
}
fname = './figpanels/PCA_leg_modules_pct.svg'
# pre_to_mn_df.head()
# mpool_dict

var_frac_dict = {}
pca_decomp = PCA(n_components=None,
        copy=True, 
        whiten=False, 
        svd_solver='full', 
        tol=0.0, 
        iterated_power='auto')


In [ ]:
local_df = pre_to_mn_df.loc['local',:]
# local_df.head()

In [ ]:
var_frac_dict = {}
pca_decomp = PCA(n_components=None,
        copy=True, 
        whiten=False, 
        svd_solver='full', 
        tol=0.0, 
        iterated_power='auto')

for pool in pool_keys:
    syn = local_df.loc[pref_pool_dict[pool],mpool_dict[pool]]
    print(syn.shape)
    mu = np.mean(syn, axis=0)

    syn_pca = pca_decomp.fit(syn)
    A0 = syn_pca.transform(syn)
    var_frac_dict[pool] = syn_pca.explained_variance_/syn_pca.explained_variance_.sum()

fig, ax = plt.subplots( 1, 1, figsize=(6,6))
clrs = sns.color_palette()
clrs = [
    '#555555',
    '#888888',
    '#A0A0A0',
    '#BFBFBF',
    '#DFDFDF',
    '#E0E0E0',
    '#E7E7E7',
    '#EBEBEB',
    '#EFEFEF',
    '#F2F2F2',
    '#F4F4F4',
    '#F8F8F8',
]

m = 0
pc1 = []
pc2 = []
for pool in pool_keys:
    pc_frac = var_frac_dict[pool]
    bottom = 0.
    pc1+=[pc_frac[0]]
    pc2+=[pc_frac[1]]
    for pc_fr,clr in zip(pc_frac,clrs):
        p = ax.bar(m, pc_fr, 0.8, bottom=bottom,color=clr)
        bottom += pc_fr
    m+=1

x = [i for i in range(len(pool_keys))]
ax.plot(x,pc1,color='#000000')
ax.plot(x,pc2,color=clrs[1])
ax.plot(x,np.array(pc2)/np.array(pc1))
ax.set_xticks(x)
ax.set_xticklabels(pool_keys)
plt.xticks(rotation=90)

fig.savefig(fname,format='svg')
fig.savefig('./figpanels/PCA_leg_modules_pct.svg',format='svg')

In [ ]:
# Plot the first 3 PCs for Tibia flexors

# mft_df = femur_df_local.loc[('local','main_tibia_flexor',All,All),mpool_dict['aux_main_tibia_flexor']]

pref_pool_dict['aux_main_tibia_flexor_wtarsus'] = 'main_tibia_flexor'

syn = local_df.loc[pref_pool_dict['aux_main_tibia_flexor_wtarsus'],mpool_dict['aux_main_tibia_flexor_wtarsus']]
print(syn.shape)
mu = np.mean(syn, axis=0)

syn_pca = pca_decomp.fit(syn)
A0 = syn_pca.transform(syn)

fig, ax = plt.subplots( 1, 1, figsize=(6,6))
ax.plot(syn_pca.components_[0,:])
ax.plot(syn_pca.components_[1,:])
ax.plot(syn_pca.components_[2,:])
fig.savefig('./figpanels/tibiaflexA_wtarsus_PCs.svg',format='svg')
